In [7]:
# [1] Imports e Configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
import time
import os

# Reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)

# Pastas de resultados
os.makedirs('results', exist_ok=True)



In [8]:
# [2] Carregar dataset do CSV já limpo
df = pd.read_csv('data/btc_limpo.csv')

# Garantir formato de datas
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Série alvo
series = df.set_index('Date')['Close'].astype(float)
print("Dados carregados:", series.index.min(), "->", series.index.max(), "| pontos =", len(series))
series.tail()


Dados carregados: 2020-06-01 00:00:00 -> 2025-05-31 00:00:00 | pontos = 1826


Date
2025-05-27    108994.640625
2025-05-28    107802.328125
2025-05-29    105641.757812
2025-05-30    103998.570312
2025-05-31    104638.093750
Name: Close, dtype: float64

In [9]:
# [3] Funções utilitárias para preparar dados
def prepare_data(series, window=60, test_size=0.1):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled = scaler.fit_transform(series.values.reshape(-1,1))

    X, y = [], []
    for i in range(window, len(scaled)):
        X.append(scaled[i-window:i, 0])
        y.append(scaled[i, 0])
    X, y = np.array(X), np.array(y)

    split = int(len(X)*(1-test_size))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # reshape para LSTM/GRU [amostras, passos, features]
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    return X_train, X_test, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = prepare_data(series, window=60)
print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


Shapes: (1589, 60, 1) (177, 60, 1) (1589,) (177,)


In [10]:
# [4] Função genérica para treinar LSTM ou GRU
def run_model(model_type='LSTM', units=50, dropout=0.2, lr=0.001, epochs=50, batch_size=32):
    tf.keras.backend.clear_session()
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(units, return_sequences=False, input_shape=(X_train.shape[1],1)))
    elif model_type == 'GRU':
        model.add(GRU(units, return_sequences=False, input_shape=(X_train.shape[1],1)))
    else:
        raise ValueError("Modelo não suportado: use 'LSTM' ou 'GRU'")
    model.add(Dropout(dropout))
    model.add(Dense(1))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mse', optimizer=optimizer)

    start = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    train_time = time.time() - start

    preds = model.predict(X_test)
    preds = scaler.inverse_transform(preds.reshape(-1,1)).flatten()
    y_true = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

    score = mape(y_true, preds)*100

    return score, train_time, preds, y_true


In [11]:
# [5] Mini Grid Search
param_grid = {
    'units': [32, 64],
    'dropout': [0.1, 0.2],
    'lr': [0.001, 0.0005],
    'epochs': [50],
}

results = []

for model_type in ['LSTM','GRU']:
    for units in param_grid['units']:
        for dropout in param_grid['dropout']:
            for lr in param_grid['lr']:
                for epochs in param_grid['epochs']:
                    score, train_time, preds, y_true = run_model(
                        model_type=model_type,
                        units=units,
                        dropout=dropout,
                        lr=lr,
                        epochs=epochs
                    )
                    results.append({
                        'Modelo': model_type,
                        'Units': units,
                        'Dropout': dropout,
                        'LR': lr,
                        'Epochs': epochs,
                        'MAPE': score,
                        'TempoTreino(s)': train_time
                    })
                    print(f"{model_type} | Units={units}, Dropout={dropout}, LR={lr} -> MAPE={score:.2f}% | Tempo={train_time:.1f}s")

# DataFrame com os resultados
df_results = pd.DataFrame(results)
df_results.to_csv('results/grid_results_02-10.csv', index=False)
df_results.sort_values('MAPE')


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step 
LSTM | Units=32, Dropout=0.1, LR=0.001 -> MAPE=2.34% | Tempo=34.4s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step 
LSTM | Units=32, Dropout=0.1, LR=0.0005 -> MAPE=2.42% | Tempo=34.1s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step 
GRU | Units=64, Dropout=0.2, LR=0.001 -> MAPE=2.28% | Tempo=55.0s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step 
GRU | Units=64, Dropout=0.2, LR=0.0005 -> MAPE=1.94% | Tempo=59.2s


,Modelo,Units,Dropout,LR,Epochs,MAPE,TempoTreino(s)
8,GRU,32,0.1,0.0010,50,1.840680,59.737756
10,GRU,32,0.2,0.0010,50,1.919779,47.019046
15,GRU,64,0.2,0.0005,50,1.938166,59.247181
12,GRU,64,0.1,0.0010,50,1.958577,58.594173
9,GRU,32,0.1,0.0005,50,1.963276,46.018914
4,LSTM,64,0.1,0.0010,50,2.099928,69.294416
6,LSTM,64,0.2,0.0010,50,2.160619,77.579817
14,GRU,64,0.2,0.0010,50,2.281521,55.035965
0,LSTM,32,0.1,0.0010,50,2.341687,34.359153
7,LSTM,64,0.2,0.0005,50,2.363413,77.424371


In [12]:
# [5] Mini Grid Search
param_grid = {
    'units': [32, 64],
    'dropout': [0.1, 0.2],
    'lr': [0.001, 0.0005],
    'epochs': [50],
}

results = []

for model_type in ['LSTM','GRU']:
    for units in param_grid['units']:
        for dropout in param_grid['dropout']:
            for lr in param_grid['lr']:
                for epochs in param_grid['epochs']:
                    score, train_time, preds, y_true = run_model(
                        model_type=model_type,
                        units=units,
                        dropout=dropout,
                        lr=lr,
                        epochs=epochs
                    )
                    results.append({
                        'Modelo': model_type,
                        'Units': units,
                        'Dropout': dropout,
                        'LR': lr,
                        'Epochs': epochs,
                        'MAPE': score,
                        'TempoTreino(s)': train_time
                    })
                    print(f"{model_type} | Units={units}, Dropout={dropout}, LR={lr} -> MAPE={score:.2f}% | Tempo={train_time:.1f}s")

# DataFrame com os resultados
df_results = pd.DataFrame(results)
df_results.to_csv('results/grid_results_02-10.csv', index=False)
df_results.sort_values('MAPE')


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step 
LSTM | Units=32, Dropout=0.1, LR=0.001 -> MAPE=2.40% | Tempo=31.0s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step 
LSTM | Units=32, Dropout=0.1, LR=0.0005 -> MAPE=2.62% | Tempo=37.9s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step 
GRU | Units=64, Dropout=0.2, LR=0.001 -> MAPE=2.27% | Tempo=43.8s


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step 
GRU | Units=64, Dropout=0.2, LR=0.0005 -> MAPE=2.57% | Tempo=42.4s


,Modelo,Units,Dropout,LR,Epochs,MAPE,TempoTreino(s)
8,GRU,32,0.1,0.0010,50,1.844563,36.664946
13,GRU,64,0.1,0.0005,50,1.965893,42.117684
12,GRU,64,0.1,0.0010,50,2.017128,48.373665
10,GRU,32,0.2,0.0010,50,2.064444,38.365172
9,GRU,32,0.1,0.0005,50,2.076117,38.876327
11,GRU,32,0.2,0.0005,50,2.233411,38.569018
14,GRU,64,0.2,0.0010,50,2.268824,43.837374
0,LSTM,32,0.1,0.0010,50,2.402542,31.046127
4,LSTM,64,0.1,0.0010,50,2.426785,45.250964
2,LSTM,32,0.2,0.0010,50,2.428205,37.367226


In [13]:
# [6] Plot do melhor modelo
best = df_results.sort_values('MAPE').iloc[0]
print("Melhor configuração encontrada:")
print(best)

score, _, preds, y_true = run_model(
    model_type=best['Modelo'],
    units=int(best['Units']),
    dropout=float(best['Dropout']),
    lr=float(best['LR']),
    epochs=int(best['Epochs'])
)

plt.figure(figsize=(12,6))
plt.plot(y_true[-200:], label='Real', color='blue')
plt.plot(preds[-200:], label='Previsto', color='red')
plt.title(f"{best['Modelo']} Melhor Config - MAPE={score:.2f}%")
plt.legend()
plt.savefig(f"results/{best['Modelo']}_best_02-10.png", dpi=300)
plt.close()
print(f"✅ Gráfico salvo em results/{best['Modelo']}_best_02-10.png")


Melhor configuração encontrada:
Modelo                  GRU
Units                    32
Dropout                 0.1
LR                    0.001
Epochs                   50
MAPE               1.844563
TempoTreino(s)    36.664946
Name: 8, dtype: object


C:\Users\guiga\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step 
✅ Gráfico salvo em results/GRU_best_02-10.png
